# Trabalho Prático 02 - Realidade Aumentada

- Giovanna Louzi Bellonia - 2017086015
- Thiago Martin Poppe - 2017014324


- __Obs.:__ Por problemas graves com a biblioteca GLUT, não conseguimos rodar o TP usando ela... Por isso, optamos por utilizar a biblioteca GLFW com versão 1.8.0. Para baixar via anaconda, basta rodar o comando conda install -c conda-forge glfw ou pelo pip usando o comando pip install glfw. Caso não consiga rodar o código por completo, enviaremos uma imagem juntamente com o notebook para mostrar o resultado.

In [1]:
import cv2
import glfw

import numpy as np

from OpenGL.GL import *
from OpenGL.GLU import *
from OpenGL.GLUT import *

from ObjLoader import OBJ

glfw.__version__

'1.8.0'

# Lendo o vídeo

In [2]:
cap = cv2.VideoCapture('entrada.avi')
frames = []

while cap.isOpened():
    ret, frame = cap.read()
    if ret == False:
        print('*** Video ended ***')
        break
        
    frames.append(frame)
    
frames = np.array(frames)

*** Video ended ***


# Função homography
- Função criada para estimar a matriz de homografia utilizando a função findHomography com o método RANSAC para filtrar erros grosseiros e a função warpPerspective para aplicar tal matriz na imagem.
- Irá retornar o resultado binarizado para realizarmos posteriormente um template matching mais preciso.

In [3]:
def homography(img, src, target):
    """
        Estima e aplica a matriz de homografia em um conjunto de pontos
        
        Parâmetros
        ----------
        img : numpy.ndarray
            Imagem de onde os pontos foram extraídos
            
        src : numpy.ndarray
            Pontos de origem (da imagem)
            
        target : numpy.ndarray
            Pontos do alvo
            
        Retorno
        -------
        numpy.ndarray
            Retorna aquela parte da imagem binarizada e com a homografia aplicada
    """
    
    # Obtendo os pontos no alvo e estimando a matriz de homografia usando RANSAC
    height, width = target.shape
    dst = np.float32([[0,0], [0, height], [width, height], [width, 0]])
    M = cv2.findHomography(src, dst, cv2.RANSAC)[0]
    
    # Aplicando o warpPerspective e binarizando a imagem
    result = cv2.warpPerspective(img, M, (width, height))
    result[result < 80] = 0
    result[result >= 80] = 255
    
    return result

# Função get_targets
- Função auxiliar usada para ler o arquivo que contêm o alvo, retornando o mesmo rotacionado em 0º, 90º, 180º e 270º respectivamente.

In [4]:
def get_targets(filename):
    """
        Retorna o alvo rotacionado em 0º, 90º, 180º e 270º
        
        Parâmetros
        ----------
        filename : str
            Nome do arquivo, .jpg por exemplo, do alvo
            
        Retorno
        -------
        list
            Lista contendo o alvo rotacionado em 0º, 90º, 180º e 270º (nessa ordem)
    """
    
    # Lendo o alvo, convertendo para escalas de cinza e binarizando
    target = cv2.imread(filename)
    target = cv2.cvtColor(target, cv2.COLOR_BGR2GRAY)
    target[target < 25] = 0
    target[target >= 25] = 255
    
    targets_list = []
    
    # Rotacionando o alvo em cada um dos ângulos e salvando em uma lista
    for angle in [0, 90, 180, 270]:
        M = cv2.getRotationMatrix2D((target.shape[1]/2, target.shape[0]/2), angle, 1)
        targets_list.append(cv2.warpAffine(target, M, (target.shape[0], target.shape[1])))
                            
    return targets_list

# Função similarity_func

- Função criada para retornar o quão similar duas imagens são.
- Fizemos um cálculo de diferença média absoluta dos pixels para tal.

In [5]:
def similarity_func(img1, img2):
    """
        Função de similaridade usando a diferença média dos pixels
        
        Parâmetros
        ----------
        img1 : numpy.ndarray
            Imagem 1 a ser comparada
            
        img2 : numpy.ndarray
            Imagem 2 a ser comparada
            
        Retorno
        -------
            Retorna o valor da diferença média dos pixels
    """
    
    return np.sum(np.abs(img1 - img2)) / (img1.size)

# Função template_matching
- Essa função cumpre o papel de: para cada imagem do alvo rotacionado, verificar o quão similar ela é com a imagem obtida no passo da homografia. Salvando o índice da imagem que possui o menor valor retornado pela função similarity_func, em outras palavras, a mais similar.
- Caso essa imagem não possua valor de similaridade menor do que a tolerância definida, não aceitamos e retornamos -1. Caso contrário, retornamos o índice encontrado.

In [6]:
def template_matching(img, targets, sim_func, tol=25):
    """
        Função que realiza o casamento de template
        
        Parâmetros
        ----------
        img : numpy.ndarray
            Imagem que se quer casar
            
        targets : numpy.ndarray list
            Lista de alvos que iremos casar
            
        simFunc : function
            Função de similaridade que aceita dois numpy.ndarray,
            realiza os cálculos e retorna um número
            
        tol : float, opcional
            Tolerância para ser um casamento válido (por padrão é 25)
            
        Retorno
        -------
        int
            Caso o valor de similaridade esteja abaixo da tolerância,
            retornamos o índice do alvo que resultou em um casamento
            bem-sucedido (seguindo a mesma ordem dos ângulos).
            Senão, retornamos -1.
    """
    
    min_similarity = sim_func(img, targets[0])
    min_pos = 0
    
    # Caminhando por cada alvo, comparando o valor de similaridade do atual e 
    # atualizando, caso necessário, as variáveis
    for i in range(1, len(targets)):
        current = sim_func(img, targets[i])
        if current < min_similarity:
            min_similarity = current
            min_pos = i
    
    return min_pos if min_similarity <= tol else -1

# Função seek_targets

- Essa função cumpre o papel de retornar os pontos dos alvos na cena. Usaremos a função findContours da OpenCV para tal, juntamente com a homography e template_matching para realizar o casamento de padrões.
- Para cada contorno encontrado, aproximaremos o mesmo por um polígono e, se esse tiver um tamanho igual a 4 faremos a homografia seguida do template matching para ele e, se retornar verdadeiro, guardaremos seus pontos no vetor "pts" e a posição em que se encontra em "pos".

In [7]:
def seek_targets(img, targets):
    """
        Função que retorna os pontos dos alvos na cena
        
        Parâmetros:
        ----------
        img : np.ndarray
            Imagem a ser analisada
            
        targets : list of np.ndarray
            Alvos no qual iremos realizar o template matching
            
        Retorno:
        -------
        list of np.ndarray and int list
            A função irá retornar os pontos dos alvos encontrados na cena e com qual alvo rotacionado o casamento
            foi bem sucedido
    """

    pts = []
    pos = []

    # Convertendo a imagem para tons de cinza
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.bilateralFilter(gray, 11, 17, 17)
    
    # Detectando bordas com Canny e usando a findContours
    canny = cv2.Canny(gray, 100, 200)
    contours, _ = cv2.findContours(canny, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    for c in contours:
        # Aproximando o contorno via um polígono 
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * peri, True)
        
        # Caso esse polígono possua 4 pontos adicionamos ele na nossa lista se a homografia retornar como correto
        if len(approx) == 4:
            result = homography(np.float32(gray), np.float32(approx), targets[0])
            match = template_matching(result, targets, similarity_func, 30)
            
            if  match != -1:
                pos.append(match)
                pts.append(approx)
                
        # Caso encontremos 3 alvos, podemos parar o loop
        if len(pts) == 3:
            break
    
    return pts, pos

# Função get_object_pts

- Função auxiliar que dado uma pose do alvo, iremos retornar o valor correto dos pontos do mesmo para a função solvePnP.

In [8]:
def get_object_pts(pos):
    """
        Função que retorna o valor correto dos pontos do objeto para a função solvePnP
        
        Parâmetros:
        ----------
        int
            Valor que indica com qual alvo o casamento foi bem sucedido (0 -> 0º, 1 -> 90º, 2 -> 180º ou 3 -> 270º)

        Retorno:
        -------
        np.ndarray
            Vetor de pontos do objeto na orientação correta
    """
    
    if pos == 0:
        return np.float32([[1,1,1], [1,-1,1], [-1,-1,1], [-1,1,1]])
    if pos == 1:
        return np.float32([[-1,-1,1], [-1,1,1], [1,1,1], [1,-1,1]])
    if pos == 2:
        return np.float32([[1,-1,1], [-1,-1,1], [-1,1,1], [1,1,1]])
    if pos == 3:
        return np.float32([[-1,1,1], [1,1,1], [1,-1,1], [-1,-1,1]])

# Função loadBackgroundTexture

- Função que utiliza OpenGL para carregar a textura da cena como background, retornando o 'id' do buffer gerado pela OpenGL.

In [9]:
def loadBackgroundTexture(img):
    '''
        Função que carrega a textura do fundo da cena para a OpenGL
        
        Parâmetros:
        ----------
        img : numpy.ndarray
            Imagem a ser usada como textura
            
        Retorno:
        -------
        int
            Id da textura criado pela OpenGL
    '''
    
    # Criando um id para a textura e habilitando
    background_id = glGenTextures(1)
    glBindTexture(GL_TEXTURE_2D, background_id)
    
    # Convertendo a imagem de BGR para RGB e realizando um 'flip'
    background = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    background = cv2.flip(background, 0)
    
    # Convertendo a imagem para uma string
    height, width, channels = background.shape
    background = np.fromstring(background.tostring(), dtype=background.dtype, count = height * width * channels)    
    background.shape = (height, width, channels)

    # Criando a textura na OpenGL
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)
    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGB, width, height, 0, GL_RGB, GL_UNSIGNED_BYTE, background)
    
    # Retornando o id
    return background_id

# Função showBackground

- Função que utiliza OpenGL para gerar o fundo da nossa cena

In [10]:
def showBackground(img):
    """
        Função que desenha o fundo da cena.
        
        Parâmetros:
        ----------
        img : np.ndarray
            Imagem a ser colocada como textura
    """
    
    # Lendo a textura
    textureId = loadBackgroundTexture(img)
    
    # Desabilitando o glDepthMask
    glDepthMask(GL_FALSE)
    
    # Definindo a projeção como ortográfica
    glMatrixMode(GL_PROJECTION)
    glPushMatrix()
    glLoadIdentity()
    gluOrtho2D(0, width, 0, height)
    
    # Habilitando a textura
    glEnable(GL_TEXTURE_2D)
    glBindTexture(GL_TEXTURE_2D, textureId)
    glMatrixMode(GL_MODELVIEW)
    glPushMatrix()
    
    # Desenhando um quadrilátero do tamanho da tela com a textura
    glBegin(GL_QUADS)
    glTexCoord2f(0, 0); glVertex2f(0, 0)
    glTexCoord2f(1, 0); glVertex2f(width, 0)
    glTexCoord2f(1, 1); glVertex2f(width, height)
    glTexCoord2f(0, 1); glVertex2f(0, height)
    glEnd()
    
    glPopMatrix()
    glMatrixMode(GL_PROJECTION)
    glPopMatrix()
    glMatrixMode(GL_MODELVIEW)
    
    # Desabilitando a textura e chamando glFlush
    # glDepthMask -> faz com que o fundo não fique na frente do Pikachu
    glBindTexture(GL_TEXTURE_2D, 0)
    glDepthMask(GL_TRUE)
    glFlush()

# Função initOpenGL

- Função auxiliar para iniciar a OpenGL

In [11]:
def initOpenGL():
    """
        Função que inicializa a OpenGL
        
        Retorno:
        -------
        Retorna o .obj carregado na memória
    """
    
    # Habilitando o uso de texturas e de profundidade
    glEnable(GL_TEXTURE_2D)
    glEnable(GL_DEPTH_TEST)
    
    # Carregando o objeto Pikachu na memória
    obj = OBJ('Pikachu.obj', swapyz=True)
    
    # Definindo o modo de projeção da cena
    glMatrixMode(GL_PROJECTION)
    glLoadIdentity()
    
    # Usando os parâmetros intrínsecos da câmera para calcular o fovy e aspect ratio
    fx = cameraParams['fc'][0]
    fy = cameraParams['fc'][1]
    fovy = 2 * np.arctan(0.5 * height / fy)*180 / np.pi
    aspect = (width*fy) / (height*fx)
    gluPerspective(fovy, aspect, 0.1, 100.0)
    
    # Definindo a cor de limpeza do fundo
    glClearColor(0.2, 0.2, 0.2, 0.0)
    
    return obj

# Função showObject

- Função que cumpre o papel de exibir o objeto (Pikachu) na cena fazendo uso da OpenGL.

In [12]:
def showObject(pts, pos):
    """
        Função que renderiza o objeto na cena
        
        Parâmetros:
        ----------
        pts : np.ndarray
            Pontos do alvo na cena
            
        pos : int
            Inteiro que indica com qual alvo estamos casando
    """
    
    dst = get_object_pts(pos)
    _, rvec, tvec = cv2.solvePnP(dst, np.float32(pts), intMatrix, cameraParams['kc'])
    rotm = cv2.Rodrigues(rvec)[0]

    m = np.array([
        [rotm[0][0], rotm[0][1], rotm[0][2], tvec[0]],
        [rotm[1][0], rotm[1][1], rotm[1][2], -tvec[1]],
        [rotm[2][0], rotm[2][1], rotm[2][2], -tvec[2]],
        [0.0, 0.0, 0.0, 1.0]
    ])

#     m = m * np.array([[ 1.0,  1.0,  1.0,  1.0],
#                       [-1.0, -1.0, -1.0, -1.0],
#                       [-1.0, -1.0, -1.0, -1.0],
#                       [ 1.0,  1.0,  1.0,  1.0]])

    m = np.transpose(m)
    
    glMatrixMode(GL_MODELVIEW)
    glLoadIdentity()
    glPushMatrix()
    glLoadMatrixf(m)
    glCallList(obj.gl_list)
    glPopMatrix()

# Função draw

- Função que irá representar o nosso loop principal na OpenGL.

In [13]:
def draw():
    """
        Função que irá representar nosso main loop na OpenGL
    """
   # Limpando os buffers
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    
    # Desenhando o fundo
    showBackground(frames[i])
    
    # Desenhando os objetos nos alvos
    pts, pos = seek_targets(frames[i], targets)
    for j in range(len(pts)):
        showObject(pts[j], pos[j])
    
    i += 1
    
    # Trocando de buffer para exibir na tela
    glutSwapBuffers()
    
    # Caso tenhamos lido todos os frames podemos retornar
    if i > len(frames):
        return

# Definindo os parâmetros intrínsecos da câmera

- Os valores da distância focal, centro óptico e coeficientes de distorção da câmera foram obtidos atráves da calibração pelo MATLAB utilizando o toolbox_calib.

In [14]:
# Definindo os parâmetros intrínsecos da câmera através da calibração pelo MATLAB
cameraParams = {
    'fc': [536.19341, 536.14756],
    'cc': [317.64968, 233.56773],
    'kc': np.array([0.07459, -0.15101, -0.00544, 0.00111, 0.00000])
}

# Definindo a matriz de parâmetros intrínsecos
intMatrix = np.array([
    [cameraParams['fc'][0], 0.0, cameraParams['cc'][0]],
    [0.0, cameraParams['fc'][1], cameraParams['cc'][1]],
    [0.0, 0.0, 1.0]
])

# Renderizando os objetos com a OpenGL

In [15]:
# Definindo o tamanho da nossa janela
width, height = 640, 480

# Iniciando o glfw
if not glfw.init():
    print('*** ERRO AO INICIAR O GLFW ***')
    
# Criando uma janela
window = glfw.create_window(width, height, 'TP2', None, None)
glfw.make_context_current(window)

# Carregando os alvos
targets = get_targets('alvo.jpg')

# Iniciando a OpenGL e carregando o objeto na memória
obj = initOpenGL()

i = 0
while not glfw.window_should_close(window):
    glfw.poll_events()
    
    # Limpando os buffers
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    
    # Desenhando o fundo
    showBackground(frames[i])
    
    # Desenhando os objetos nos alvos
    pts, pos = seek_targets(frames[i], targets)
    for j in range(len(pts)):
        showObject(pts[j], pos[j])
    
    i += 1
    
    # Caso tenhamos lido todos os frames podemos retornar
    if i >= len(frames):
        break
    
    glfw.swap_buffers(window)
    
# Encerrando o GLFW
glfw.terminate()

D:\Anaconda\envs\ICV\lib\site-packages\ipykernel_launcher.py:26: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


# Conclusões

- Observamos que a orientação do alvo da esquerda foi estimada de maneira errada. Como no TP1 ela foi estimada de forma certa usando o nosso método de captura dos pontos dos alvos no sentido anti-horário, acreditamos que a função findContours da OpenCV retorna os pontos em uma ordem diferente da que estavamos esperando. Resultando assim, em um casamento errado ao passarmos esses pontos pelo nosso pipeline. Por questões de tempo, não conseguimos resolver esses pequenos detalhes de implementação, porém, acreditamos que o erro está na ordem em que os pontos foram passados para as funções da OpenCV.


- Caso não consiga rodar o código por algum motivo, deixaremos prints de vários frames mostrando os resultados!